# particiones

In [1]:
import geopandas as gpd
import pandas as pd
import os

# Rutas de entrada para las bases de datos
ruta_manzanas = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC.gpkg"
ruta_buffers = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020_B.gpkg"

# Capa específica dentro del Geopackage de manzanas
capa_manzanas = 'Mzn_SubTipologias_eq'

# Categorías de búffers
categorias_buffers = ['Educación', 'Salud', 'Recreación', 'Servicios']

# Rutas de salida
output_dir_manzanas = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C"
output_dir_buffers = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Particiones_C"

os.makedirs(output_dir_manzanas, exist_ok=True)
os.makedirs(output_dir_buffers, exist_ok=True)

# Cargar la capa de manzanas
print("Cargando la capa de manzanas...")
manzanas = gpd.read_file(ruta_manzanas, layer=capa_manzanas)

# Cargar las capas de búffers para cada categoría
print("Cargando las capas de búffers...")
buffers = {}
for categoria in categorias_buffers:
    layer_name = f"{categoria}_Buffer_C"
    buffers[categoria] = gpd.read_file(ruta_buffers, layer=layer_name)

# Verificar que la columna 'CVE_ENT' existe
if 'CVE_ENT' not in manzanas.columns:
    raise ValueError("La base de datos de manzanas no contiene la columna 'CVE_ENT'.")

for categoria, buffer_data in buffers.items():
    if 'CVE_ENT' not in buffer_data.columns:
        raise ValueError(f"La capa de búffers '{categoria}' no contiene la columna 'CVE_ENT'.")

# Calcular el número total de registros por estado
manzanas_counts = manzanas['CVE_ENT'].value_counts().sort_index()

buffers_counts = {
    categoria: data['CVE_ENT'].value_counts().sort_index() for categoria, data in buffers.items()
}

# Crear un DataFrame para asignar los estados a particiones
estado_counts = pd.DataFrame({
    'CVE_ENT': manzanas_counts.index,
    'manzanas_count': manzanas_counts.values
})

for categoria in categorias_buffers:
    estado_counts[f'buffers_{categoria}_count'] = buffers_counts[categoria].reindex(estado_counts['CVE_ENT'], fill_value=0).values

estado_counts['total_count'] = estado_counts.iloc[:, 1:].sum(axis=1)
estado_counts = estado_counts.sort_values('total_count', ascending=False)

# Crear particiones equilibradas
particiones = [[] for _ in range(16)]
particiones_sums = [0] * 16

for _, row in estado_counts.iterrows():
    min_index = particiones_sums.index(min(particiones_sums))
    particiones[min_index].append(row['CVE_ENT'])
    particiones_sums[min_index] += row['total_count']

# Exportar las particiones
print("Exportando particiones...")
for i, particion in enumerate(particiones):
    # Filtrar las manzanas para la partición
    manzanas_particion = manzanas[manzanas['CVE_ENT'].isin(particion)]
    
    # Guardar las manzanas en un geopackage
    manzanas_output_path = os.path.join(output_dir_manzanas, f"Mzn_Tipologías_NAC_{i + 1}.gpkg")
    manzanas_particion.to_file(manzanas_output_path, layer='Mzn_SubTipologias_eq', driver="GPKG")
    
    # Filtrar y guardar cada capa de búffers para la partición
    buffers_output_path = os.path.join(output_dir_buffers, f"Equipamientos_NAC_2020_{i + 1}.gpkg")
    for categoria in categorias_buffers:
        buffer_particion = buffers[categoria][buffers[categoria]['CVE_ENT'].isin(particion)]
        buffer_particion.to_file(buffers_output_path, layer=f"{categoria}_Buffer_C", driver="GPKG")
    
    print(f"Partición {i + 1}:")
    print(f"  - Manzanas guardadas en: {manzanas_output_path}")
    print(f"  - Buffers guardados en: {buffers_output_path}")

print("Proceso completado.")


Cargando la capa de manzanas...
Cargando las capas de búffers...
Exportando particiones...
Partición 1:
  - Manzanas guardadas en: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\Mzn_Tipologías_NAC_1.gpkg
  - Buffers guardados en: Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Particiones_C\Equipamientos_NAC_2020_1.gpkg
Partición 2:
  - Manzanas guardadas en: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\Mzn_Tipologías_NAC_2.gpkg
  - Buffers guardados en: Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Particiones_C\Equipamientos_NAC_2020_2.gpkg
Partición 3:
  - Manzanas guardadas en: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\Mzn_Tipologías_NAC_3.gpkg
  - Buffers guardados en: Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Particiones_C\Equipamientos_NAC_2020_3.gpkg
Partición 4:
  - Manzanas guardadas en: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\Mzn_Tipologías_NAC_4.gpkg
  - Buffers guardados en: Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Particiones_C\Equipamientos_NAC_2

# Reintegrar

In [3]:
import geopandas as gpd
import pandas as pd
import os

In [5]:
# Rutas de las particiones procesadas
rutas_particiones = [
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part1_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part2_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part3_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part4_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part5_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part6_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part7_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part8_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part9_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part10_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part11_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part12_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part13_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part14_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part15_ImpEq.gpkg",
    r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part16_ImpEq.gpkg"
]

# Nombre de la capa dentro de cada geopackage
nombre_capa = "Mzn_SubTipologias_UeEq"

# Lista para almacenar los DataFrames geoespaciales
particiones = []

# Cargar cada partición y añadirla a la lista
for ruta in rutas_particiones:
    if os.path.exists(ruta):
        print(f"Cargando partición: {ruta}")
        particion = gpd.read_file(ruta, layer=nombre_capa)
        particiones.append(particion)
    else:
        print(f"No se encontró el archivo: {ruta}")

# Verificar que se cargaron todas las particiones
if len(particiones) != len(rutas_particiones):
    print("Algunas particiones no se pudieron cargar. Verifica las rutas.")
else:
    print("Todas las particiones se cargaron exitosamente.")

Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part1_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part2_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part3_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part4_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part5_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part6_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part7_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part8_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Particiones_C\RESULTADOS_C\Mzn_Part9_ImpEq.gpkg
Cargando partición: Z:\VOCES\DATA\Tra

In [6]:
# Concatenar todas las particiones en un único GeoDataFrame
manzanas_completas = gpd.GeoDataFrame(pd.concat(particiones, ignore_index=True))

# Verificar la estructura final
print(f"Total de registros en el archivo combinado: {len(manzanas_completas)}")
print(f"Estructura de datos:\n{manzanas_completas.info()}")

# Exportar el archivo combinado a un nuevo GeoPackage
ruta_salida = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC_ImpEq_BufferC.gpkg"
manzanas_completas.to_file(ruta_salida, layer='Mzn_Subtipologias_UeEQ', driver="GPKG")

print(f"Archivo integrado guardado en: {ruta_salida}")

Total de registros en el archivo combinado: 1200165
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1200165 entries, 0 to 1200164
Columns: 147 entries, CVEGEO to geometry
dtypes: float64(39), geometry(1), int64(32), object(75)
memory usage: 1.3+ GB
Estructura de datos:
None
Archivo integrado guardado en: Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC_ImpEq_BufferC.gpkg
